# Hotel Reservation Chatbot Using Deep Learning

This project implements a hotel reservation chatbot using a deep learning model built with TensorFlow and Keras. The chatbot is trained on a dataset containing various intents related to hotel bookings and interactions, enabling it to respond intelligently to user inquiries.

### Difference from the Previous Implementation

In this version of the chatbot, we used a Sequential model provided by Keras, which is a high-level neural network API that allows us to easily build and train models. This approach is different from the previous one where we implemented a **custom Simple Recurrent Neural Network (RNN)**. 

While the custom RNN implementation gave us more control over the model architecture, training, and backpropagation, it was more complex and involved manual coding for key operations such as the forward and backward passes. 

In contrast, using Keras and the Sequential model provides a more streamlined, efficient, and accessible way to design and train the chatbot’s neural network. The Keras Sequential API is designed for simpler models, especially when the model is built layer by layer in a linear stack. It allows us to leverage built-in layers like **Embedding**, **GlobalAveragePooling1D**, and **Dense**, which simplifies the process of defining and training the model. 

### Why Use Keras and TensorFlow?

- **Ease of Use**: The Sequential API is easy to understand and use, making it more suitable for rapid prototyping and deployment.
- **Efficiency**: Keras and TensorFlow offer highly optimized operations and efficient training algorithms.
- **Flexibility**: Despite being high-level, Keras also allows for easy customization if needed, providing a good balance of abstraction and flexibility.
  
This implementation allows us to focus on improving the chatbot's functionality without getting bogged down by low-level details, which is ideal for production environments where we need fast iteration and reliable performance.

This part of the code imports the necessary libraries and modules required to build, train, and evaluate the chatbot using a machine learning model. Here's a description of each library/module:

1. **`json`**:  
   The `json` module is used to work with JSON data, including reading and writing JSON files. In this case, it's used to load the chatbot's dataset (`intents.json`), which is stored in JSON format.

2. **`numpy`**:  
   `numpy` is a popular library for numerical computing in Python. It provides support for large, multi-dimensional arrays and matrices, along with a collection of mathematical functions to operate on these arrays. It is used in this project for handling data structures such as arrays and matrices during preprocessing and model training.

3. **`tensorflow` and `keras`**:  
   - **`tensorflow`** is an open-source machine learning framework developed by Google. It provides tools for building and deploying machine learning models.
   - **`keras`** is a high-level neural networks API, written in Python, and is now part of TensorFlow. It simplifies the process of building and training deep learning models. In this code, Keras is used to define and train the neural network model for the chatbot.

4. **`Sequential`**:  
   `Sequential` is a class in Keras that allows for the easy construction of models where the layers are stacked in a linear sequence. It is used to create a linear model by adding layers one by one.

5. **`Dense`, `Embedding`, `GlobalAveragePooling1D`**:  
   - **`Dense`** is a fully connected neural network layer, often used as the final layer in a model. It connects every neuron in the layer to every neuron in the previous layer.
   - **`Embedding`** is a layer that converts integer-encoded words into dense vectors of fixed size, typically used to represent words in a lower-dimensional space.
   - **`GlobalAveragePooling1D`** is a pooling layer that calculates the average of the entire sequence along the time dimension, which helps reduce the dimensionality of the model while retaining important features.

6. **`Tokenizer` and `pad_sequences` from `tensorflow.keras.preprocessing.text`**:  
   - **`Tokenizer`** is a Keras utility that helps tokenize text, i.e., convert text into sequences of integers, each representing a word or a token. This is useful for text preprocessing before feeding the data into the model.
   - **`pad_sequences`** is used to ensure that all sequences have the same length by padding shorter sequences with zeros (or another constant value) at the beginning or end, ensuring uniform input length for the model.

7. **`LabelEncoder` from `sklearn.preprocessing`**:  
   `LabelEncoder` is a part of the scikit-learn library, and it is used to encode categorical labels (such as the intent tags) into numeric values. This is needed because machine learning models work with numerical values, not textual labels.

In summary, these libraries and modules provide the essential functionality for data preprocessing, model building, training, and evaluation in the chatbot application.

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

C:\Users\nisha\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


This part of the code is responsible for loading the `intents.json` dataset, which contains the patterns (user inputs), tags (categories or intents), and the corresponding responses (bot replies). Here's the description of what happens in this code:

1. **Opening the JSON File**:  
   The code uses Python's built-in `open()` function to open the `intents2.json` file located at the specified file path. The `encoding='utf-8'` ensures that the file is read using UTF-8 encoding, which is commonly used to handle special characters or non-ASCII text in the file.

2. **Loading the Data**:  
   The `json.load()` function is used to parse the content of the JSON file into a Python dictionary. The `data` variable now holds the entire content of the `intents2.json` file as a dictionary, where the structure of the data is typically a list of intents, each containing:
   - `tag`: The label associated with the intent.
   - `patterns`: A list of possible user inputs related to that intent.
   - `responses`: A list of responses that the chatbot will use when the intent is recognized.

In essence, this code reads the `intents2.json` file and converts its content into a Python dictionary, which will be used in subsequent steps to train and test the chatbot model.

In [2]:
with open('C:/Users/nisha/Downloads/CHATBOT-20241115T051416Z-001/CHATBOT/intents2.json', encoding='utf-8') as data_file:
    data = json.load(data_file)

This section of the code processes the `intents.json` file to extract relevant information for training the chatbot model. It organizes the sentences, labels (tags), responses, and the list of unique tags for the classification task. Here's a step-by-step breakdown:

1. **Initialization of Lists**:  
   - `training_sentences`: This list will store all the user input patterns (e.g., questions or statements) that the chatbot will learn from.
   - `training_labels`: This list will store the corresponding tags (labels) for each user input, which are used to categorize the inputs into different classes.
   - `labels`: This list will store all unique tags present in the dataset, ensuring that each tag is only stored once.
   - `responses`: This list will store the predefined responses associated with each tag, which the chatbot will use to reply based on the predicted tag.

2. **Looping Through Intents**:  
   The code iterates through each intent in the dataset:
   - **Extracting Patterns and Tags**:  
     Each intent contains multiple `patterns` (phrases or questions a user might ask), and each pattern is associated with a `tag` (which is essentially the intent's label). The code appends each pattern to `training_sentences` and the associated tag to `training_labels`.
     
   - **Handling Responses**:  
     Each intent may also contain a list of `responses` (possible chatbot replies for that intent). If the `responses` key exists for the current intent, it is appended to the `responses` list. If it does not exist, an empty list is added (to avoid errors during processing).

   - **Storing Unique Tags**:  
     For each intent, the tag is added to the `labels` list if it is not already present. This ensures that the `labels` list will only contain unique tags from all intents.

3. **Calculating Number of Classes**:  
   After collecting all the unique tags, the `num_classes` variable is calculated by finding the length of the `labels` list. This represents the total number of unique intents that the chatbot needs to classify, which will be used for the output layer of the neural network model.

In summary, this section of the code processes the intent data to extract the necessary components (patterns, tags, and responses) for training the chatbot. It prepares the input data and labels for further processing in the machine learning pipeline.

In [5]:
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    
    # Check if 'responses' key exists before appending
    if 'responses' in intent:
        responses.append(intent['responses'])
    else:
        responses.append([])  # Append an empty list if 'responses' is missing
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)


This section of the code is responsible for encoding the labels (or tags) associated with each training sentence into numerical values, which is a necessary step when working with machine learning models that require numerical input.

1. **LabelEncoder Initialization**:  
   The `LabelEncoder` from the `sklearn.preprocessing` module is initialized. This encoder is used to convert categorical labels (in this case, the tags associated with each sentence) into numerical values.

2. **Fitting the Encoder**:  
   The `fit()` method is called on the `LabelEncoder` with the list of training labels (`training_labels`). This step allows the encoder to learn the unique tags in the dataset and assign each tag a unique integer. The encoder essentially maps each tag to an index, e.g., if the tags are ['greeting', 'goodbye', 'information'], they might be mapped to [0, 1, 2].

3. **Transforming Labels**:  
   After fitting, the `transform()` method is applied to the `training_labels`. This converts the original categorical tags (such as 'greeting', 'goodbye', etc.) into their corresponding numerical labels (e.g., 0, 1, 2). The `training_labels` list is now converted from a list of strings to a list of integers, which can be fed into the neural network model for classification.

In summary, this process ensures that the categorical labels (tags) in the dataset are transformed into a numerical format that the model can understand, enabling the model to perform classification tasks.

In [6]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


This section of the code is responsible for preprocessing the text data, specifically the user input sentences, before feeding them into the neural network model. Here's a breakdown of each step:

1. **Vocabulary and Sequence Setup**:  
   The vocabulary size (`vocab_size`) is set to 1000, which means only the top 1000 most frequent words in the dataset will be considered. Any words that are less frequent will be ignored. This helps in reducing the complexity of the model. Additionally, an "out-of-vocabulary" token (`oov_token`) is introduced to handle words that are not found in the training data.

2. **Tokenization**:  
   The `Tokenizer` is used to convert each word in the sentences into a unique integer. This step transforms text into numerical sequences, where each word corresponds to an index based on its frequency in the training data. The tokenizer is trained using the provided training sentences, allowing it to learn the frequency of each word and create a word index.

3. **Sequence Transformation**:  
   After the tokenizer is trained, the sentences are transformed into sequences of integers. Each word in the sentence is replaced with its corresponding integer from the word index. If a word is not found in the index, it is replaced with the `<OOV>` token.

4. **Padding and Truncating Sequences**:  
   Since different sentences have different lengths, padding and truncating are applied to make all sequences the same length. In this case, the maximum sequence length is set to 20. Sentences shorter than this length are padded with zeros, while sentences longer than this length are truncated from the end. This ensures that all input sequences have the same shape, which is required for feeding them into a neural network.

These steps collectively ensure that the text data is transformed into a suitable format (numerical sequences of consistent length) for input to the machine learning model, facilitating the model's training and inference processes.

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

### Description of Code

This part of the code defines a machine learning model using Keras, a high-level API for building neural networks. The model architecture is as follows:

1. **`Sequential()`**:  
   The model is instantiated using Keras' `Sequential` class, which allows for the addition of layers in a linear stack. Each layer has one input and one output, and they are stacked sequentially from input to output.

2. **`Embedding(vocab_size, embedding_dim, input_length=max_len)`**:  
   - This is the **embedding layer**. It converts the input tokens (integers representing words) into dense vectors of fixed size.
   - `vocab_size`: The size of the vocabulary (total number of unique words in the dataset).
   - `embedding_dim`: The dimension of the dense word embeddings (a hyperparameter, often chosen based on experimentation).
   - `input_length=max_len`: The maximum length of the input sequences, ensuring that all sequences have the same length.

3. **`GlobalAveragePooling1D()`**:  
   This is a **pooling layer** that applies **global average pooling** across the time dimension (sequence length). It computes the average of all the word embeddings in a sequence, reducing the sequence to a fixed-size vector representation, regardless of the sequence length. This helps reduce the model's complexity by lowering the dimensionality of the data.

4. **`Dense(16, activation='relu')`**:  
   A **fully connected (dense) layer** with 16 neurons and the **ReLU** activation function. The ReLU function (Rectified Linear Unit) introduces non-linearity and helps the model learn complex patterns. This layer is used twice to increase the model's capacity to learn from the data.

5. **`Dense(num_classes, activation='softmax')`**:  
   The final output layer is also a **dense layer** with a number of units equal to the number of output classes (i.e., intent categories).
   - **Softmax activation** is applied, which normalizes the output so that the values are probabilities that sum to 1. Each value represents the model’s confidence that a given input belongs to a particular class (intent).

6. **`model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])`**:  
   - **`loss='sparse_categorical_crossentropy'`**: The loss function used here is sparse categorical cross-entropy, which is appropriate for multi-class classification problems where the target labels are integers (rather than one-hot encoded).
   - **`optimizer='adam'`**: The Adam optimizer is used for updating the model weights during training. Adam is a widely used optimizer due to its adaptive learning rate and efficiency.
   - **`metrics=['accuracy']`**: The model will track accuracy during training, which will help evaluate the model's performance.

In summary, this model architecture is designed for text classification, where the goal is to predict an intent (category) for a given user input. The architecture uses embedding to convert words into vectors, followed by a pooling layer and several dense layers for classification. The model is compiled with a sparse categorical cross-entropy loss function, and the Adam optimizer is used for training.

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\nisha\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
pip install --upgrade tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Model Summary

The model summary provides detailed information about the architecture of the neural network, including the layers, their output shapes, the number of parameters, and whether the parameters are trainable.

Here’s the breakdown of each layer:

1. **Embedding Layer**:
   - **Output Shape**: `(None, 20, 16)`
   - **Parameters**: 16,000
   - The embedding layer converts input sentences into dense vectors of size 16. The input length is fixed at 20, which means the model expects sequences of 20 words or less. If a sentence is shorter than 20 words, it will be padded.
   - **Total parameters**: 16,000 (The embedding matrix has 1,000 unique words in the vocabulary, each represented by a 16-dimensional vector, so the number of parameters is 1,000 * 16).

2. **GlobalAveragePooling1D Layer**:
   - **Output Shape**: `(None, 16)`
   - **Parameters**: 0
   - This layer applies global average pooling on the sequence of word embeddings, reducing the sequence from a length of 20 to a single vector of length 16 (the number of features in the embeddings). The number of parameters is 0 because this layer does not have any weights to train.

3. **Dense Layer (First Dense)**:
   - **Output Shape**: `(None, 16)`
   - **Parameters**: 272
   - This fully connected layer has 16 neurons and applies the ReLU activation function. It takes in the 16-dimensional vector from the pooling layer. The number of parameters here is calculated as `(input_size * number_of_neurons) + number_of_neurons`, which is `(16 * 16) + 16 = 272`.

4. **Dense Layer (Second Dense)**:
   - **Output Shape**: `(None, 16)`
   - **Parameters**: 272
   - This is another fully connected layer with 16 neurons, following the same structure as the previous dense layer.

5. **Dense Layer (Output Layer)**:
   - **Output Shape**: `(None, 15)`
   - **Parameters**: 255
   - This final dense layer has 15 neurons (equal to the number of classes in the intent classification task), and it uses the **softmax** activation function to output probabilities for each of the possible classes. The number of parameters here is `(input_size * number_of_neurons) + number_of_neurons`, which is `(16 * 15) + 15 = 255`.

### Total Parameters:
- **Total Parameters**: 50,399 (196.88 KB)
- **Trainable Parameters**: 16,799 (65.62 KB)
- **Non-Trainable Parameters**: 0 (0.00 B)

The total number of parameters is 50,399, but only 16,799 are trainable because the embedding weights and the dense layer weights are trainable, while the pooling layer has no weights to train.

### Optimizer Params:
- **Optimizer Params**: 33,600 (131.25 KB)
- The optimizer parameters are separate from the model's trainable parameters. These are used by the Adam optimizer to update the model weights during training.

In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 16)              │          16,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 16)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15)                  │             255 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,399 (196.88 KB)

 Trainable params: 16,799 (65.62 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 33,600 (131.25 KB)

### Training the Model

1. **`model.fit()`**:
   - This method is used to train the model on the data. It takes in the input data (`padded_sequences`), the corresponding target labels (`np.array(training_labels)`), and the number of epochs for which to train the model.
   - **Input Data (`padded_sequences`)**: The preprocessed and padded sequences of user inputs, which are the training sentences represented as sequences of integers (tokenized words).
   - **Target Data (`np.array(training_labels)`)**: The labels (intents) corresponding to the training sentences, encoded as integers.
   - **Epochs**: The number of times the entire dataset is passed through the model during training. In this case, the model will train for 550 epochs. This means the model will go through the entire dataset 550 times, adjusting the weights after each epoch.

2. **`history`**:
   - The result of `model.fit()` is stored in the `history` variable. This contains information about the training process, including the loss and accuracy metrics at each epoch. It is useful for plotting the training progress and evaluating the model's performance over time.

#### Key Points:

- **Epochs (550)**: The model will undergo 550 iterations over the entire dataset, updating its weights after each pass.
- **Training Process**: During each epoch, the model uses the input data (`padded_sequences`) to predict labels and compare them with the true labels (`training_labels`). The loss function calculates the error, and the optimizer (Adam) adjusts the model’s weights accordingly.
  
This step will train the model to learn the relationships between the user input and its corresponding intent labels, enabling it to classify new user inputs into one of the predefined categories (intents).

In [13]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0607 - loss: 2.7106
Epoch 2/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0450 - loss: 2.7080   
Epoch 3/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0607 - loss: 2.7059
Epoch 4/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2080 - loss: 2.7051
Epoch 5/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1820 - loss: 2.7040
Epoch 6/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1698 - loss: 2.7026
Epoch 7/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2466 - loss: 2.7007
Epoch 8/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2119 - loss: 2.7005 
Epoch 9/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2084 - loss: 2.6995
Epoch 10/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1388 - loss: 2.6977 
Epoch 11/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0685 - loss: 2.6968
Epoch 12/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0528 - loss: 

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1742 - loss: 2.5215
Epoch 71/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2725 - loss: 2.4789 
Epoch 72/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2872 - loss: 2.5229
Epoch 73/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3302 - loss: 2.4597
Epoch 74/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2872 - loss: 2.4999
Epoch 75/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3146 - loss: 2.4764
Epoch 76/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2808 - loss: 2.4506
Epoch 77/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2040 - loss: 2.4971
Epoch 78/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1737 - loss: 2.4608
Epoch 79/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1932 - loss: 2.4494
Epoch 80/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2348 - loss: 2.4176
Epoch 81/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2803 - loss: 2.4709
Epo

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3302 - loss: 2.1671
Epoch 140/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2872 - loss: 2.2432  
Epoch 141/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2755 - loss: 2.2342
Epoch 142/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3380 - loss: 2.0921
Epoch 143/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2911 - loss: 2.1937
Epoch 144/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3362 - loss: 2.2059
Epoch 145/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4237 - loss: 2.1634 
Epoch 146/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4315 - loss: 2.1562 
Epoch 147/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3410 - loss: 2.1843
Epoch 148/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2950 - loss: 2.1307
Epoch 149/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3146 - loss: 2.0880
Epoch 150/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3146 - loss

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3146 - loss: 1.8298
Epoch 208/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2989 - loss: 1.8263
Epoch 209/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3380 - loss: 1.7577
Epoch 210/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3341 - loss: 1.7522
Epoch 211/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2677 - loss: 1.8959
Epoch 212/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2989 - loss: 1.8401
Epoch 213/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3068 - loss: 1.7714
Epoch 214/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3185 - loss: 1.7732
Epoch 215/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.3302 - loss: 1.7564 
Epoch 216/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2950 - loss: 1.8169
Epoch 217/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3571 - loss: 1.7719 
Epoch 218/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4085 - loss: 

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5788 - loss: 1.5095 
Epoch 276/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5949 - loss: 1.4649
Epoch 277/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6023 - loss: 1.4738
Epoch 278/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6140 - loss: 1.4540
Epoch 279/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5558 - loss: 1.4737
Epoch 280/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5910 - loss: 1.4421 
Epoch 281/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6105 - loss: 1.3773
Epoch 282/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5528 - loss: 1.4515
Epoch 283/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5715 - loss: 1.4457 
Epoch 284/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6062 - loss: 1.4262
Epoch 285/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6057 - loss: 1.4591
Epoch 286/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6222 - loss

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7526 - loss: 1.06085
Epoch 344/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7999 - loss: 1.0475
Epoch 345/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7647 - loss: 1.1160
Epoch 346/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7652 - loss: 1.0528
Epoch 347/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7500 - loss: 1.0495
Epoch 348/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7613 - loss: 1.0511
Epoch 349/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7730 - loss: 1.0436
Epoch 350/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7877 - loss: 1.0479
Epoch 351/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7955 - loss: 1.0082
Epoch 352/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8180 - loss: 1.0210
Epoch 353/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8102 - loss: 1.0421
Epoch 354/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8415 - loss:

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8405 - loss: 0.77612
Epoch 412/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8596 - loss: 0.7345
Epoch 413/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8909 - loss: 0.7420
Epoch 414/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8562 - loss: 0.7228
Epoch 415/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8732 - loss: 0.7592
Epoch 416/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8869 - loss: 0.7207
Epoch 417/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8518 - loss: 0.7889
Epoch 418/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8752 - loss: 0.7189 
Epoch 419/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8909 - loss: 0.7003
Epoch 420/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8631 - loss: 0.7546
Epoch 421/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8899 - loss: 0.7373
Epoch 422/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8713 - loss

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8899 - loss: 0.5398
Epoch 480/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.8899 - loss: 0.5437 
Epoch 481/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8977 - loss: 0.5351
Epoch 482/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8821 - loss: 0.5356
Epoch 483/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8899 - loss: 0.5195
Epoch 484/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9095 - loss: 0.5122
Epoch 485/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8821 - loss: 0.5604  
Epoch 486/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9090 - loss: 0.4985 
Epoch 487/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9129 - loss: 0.5142
Epoch 488/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9090 - loss: 0.4933 
Epoch 489/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9281 - loss: 0.4791
Epoch 490/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8977 - los

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9472 - loss: 0.3524 
Epoch 548/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9315 - loss: 0.3884
Epoch 549/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9393 - loss: 0.3695
Epoch 550/550
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9393 - loss: 0.3594 


### Saving the Model and Preprocessing Objects

This section of the code is responsible for saving the trained model and essential preprocessing components (the tokenizer and label encoder) to disk, allowing the chatbot to be used later without requiring retraining.

1. **Saving the Model**:  
   The trained model is saved in the HDF5 format (`.h5`). This file contains all the details about the model, including its architecture, learned weights, and training configurations. Saving the model in this format ensures that it can be reloaded and used for making predictions in future sessions without needing to retrain.

2. **Saving the Tokenizer**:  
   The tokenizer, which is responsible for converting words into numerical representations (word indices), is saved to a file using the `pickle` module. This allows for consistent text preprocessing when the model is reloaded, ensuring that any input text is tokenized in the same way as during training.

3. **Saving the Label Encoder**:  
   The label encoder, which is used to transform categorical labels (such as different intents) into numeric values, is also saved to a file using `pickle`. This ensures that when the model is reloaded, the same label encoding is used to map predictions back to the correct intent labels.

By saving the model, tokenizer, and label encoder, the system can be efficiently restored, and the chatbot can continue to make predictions without needing to retrain or reprocess the input data.

In [14]:
# saving model
model.save("chat_model.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)